# Reddit r/news Veri Seti Analizi (2008-2024)

## Veri Bilimine Giriş Dersi Projesi

Bu notebook, Reddit r/news subreddit'inden 2008-2024 yılları arasındaki verileri analiz etmektedir.

### Analiz Hedefleri:
1. Veri setini keşfetme ve anlama
2. Zaman serisi analizleri
3. Metin madenciliği ve kelime analizi
4. Etkileşim paternlerini inceleme
5. Görselleştirme ve bulgular

## 1. Kütüphaneleri İçe Aktarma

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import plotly.express as px
import plotly.graph_objects as go

import re
from collections import Counter
import nltk

from datetime import datetime
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✓ Kütüphaneler başarıyla yüklendi!")

## 2. Veri Setini Yükleme

In [ ]:

import kagglehub

path = kagglehub.dataset_download("bwandowando/reddit-rnews-subreddit-2008-to-2024")
print(f"Veri seti konumu: {path}")

import os
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
print(f"\nBulunan CSV dosyaları: {csv_files}")

if csv_files:
    data_file = os.path.join(path, csv_files[0])
    print(f"\nYükleniyor: {csv_files[0]}")
    df = pd.read_csv(data_file)
    print(f"✓ Veri seti başarıyla yüklendi! Toplam {len(df):,} satır")
else:
    print("Hata: CSV dosyası bulunamadı!")

## 3. Veri Setini İnceleme (EDA - Exploratory Data Analysis)

In [ ]:
print("İlk 5 satır:")
df.head()

In [ ]:
print("Veri Seti Bilgileri:")
print("="*60)
df.info()

In [ ]:
print("Sütunlar:")
print(df.columns.tolist())

In [ ]:
print("Sayısal Sütunlar İçin İstatistikler:")
df.describe()

In [ ]:
print("Eksik Değerler:")
print("="*60)
missing = df.isnull().sum()
missing_pct = 100 * df.isnull().sum() / len(df)
missing_table = pd.concat([missing, missing_pct], axis=1, keys=['Toplam', 'Yüzde (%)'])
print(missing_table[missing_table['Toplam'] > 0].sort_values('Toplam', ascending=False))

## 4. Veri Ön İşleme

In [ ]:

date_columns = [col for col in df.columns if 'date' in col.lower() or 'time' in col.lower() or 'created' in col.lower()]
print(f"Tarih ile ilgili sütunlar: {date_columns}")

if 'created_utc' in df.columns:
    df['created_utc'] = pd.to_datetime(df['created_utc'], unit='s')
    df['year'] = df['created_utc'].dt.year
    df['month'] = df['created_utc'].dt.month
    df['day'] = df['created_utc'].dt.day
    df['hour'] = df['created_utc'].dt.hour
    df['day_of_week'] = df['created_utc'].dt.dayofweek
    df['day_name'] = df['created_utc'].dt.day_name()
    print("✓ Tarih sütunları oluşturuldu")

if 'title' in df.columns:
    df['title_length'] = df['title'].str.len()
    df['title_word_count'] = df['title'].str.split().str.len()
    print("✓ Başlık özellikleri oluşturuldu")

print(f"\n✓ Veri ön işleme tamamlandı! Yeni boyut: {df.shape}")

## 5. Temel İstatistikler

In [ ]:
print("📊 TEMEL İSTATİSTİKLER")
print("="*60)
print(f"Toplam gönderi sayısı: {len(df):,}")
print(f"Tarih aralığı: {df['created_utc'].min()} - {df['created_utc'].max()}")
print(f"Toplam süre: {(df['created_utc'].max() - df['created_utc'].min()).days} gün")

if 'score' in df.columns:
    print(f"\nOrtalama score: {df['score'].mean():.2f}")
    print(f"Medyan score: {df['score'].median():.2f}")
    print(f"Maksimum score: {df['score'].max():,}")

if 'num_comments' in df.columns:
    print(f"\nOrtalama yorum sayısı: {df['num_comments'].mean():.2f}")
    print(f"Toplam yorum: {df['num_comments'].sum():,}")

if 'title_length' in df.columns:
    print(f"\nOrtalama başlık uzunluğu: {df['title_length'].mean():.2f} karakter")
    print(f"Ortalama kelime sayısı: {df['title_word_count'].mean():.2f} kelime")

## 6. Zaman Serisi Analizleri

In [ ]:
yearly_posts = df.groupby('year').size().reset_index(name='post_count')

plt.figure(figsize=(14, 6))
plt.bar(yearly_posts['year'], yearly_posts['post_count'], color='steelblue', alpha=0.8)
plt.title('Yıllara Göre Gönderi Sayısı (2008-2024)', fontsize=16, fontweight='bold')
plt.xlabel('Yıl', fontsize=12)
plt.ylabel('Gönderi Sayısı', fontsize=12)
plt.xticks(yearly_posts['year'], rotation=45)
plt.grid(axis='y', alpha=0.3)

for i, v in enumerate(yearly_posts['post_count']):
    plt.text(yearly_posts['year'].iloc[i], v, f'{v:,}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\nYıllık İstatistikler:")
print(yearly_posts)

In [ ]:
monthly_activity = df.groupby('month').size().reset_index(name='post_count')
month_names = ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 
               'Temmuz', 'Ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık']
monthly_activity['month_name'] = monthly_activity['month'].apply(lambda x: month_names[x-1])

plt.figure(figsize=(12, 6))
plt.plot(monthly_activity['month'], monthly_activity['post_count'], 
         marker='o', linewidth=2, markersize=8, color='coral')
plt.title('Aylara Göre Gönderi Dağılımı', fontsize=16, fontweight='bold')
plt.xlabel('Ay', fontsize=12)
plt.ylabel('Toplam Gönderi Sayısı', fontsize=12)
plt.xticks(range(1, 13), month_names, rotation=45)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
day_activity = df.groupby('day_name').size().reset_index(name='post_count')
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_names_tr = ['Pazartesi', 'Salı', 'Çarşamba', 'Perşembe', 'Cuma', 'Cumartesi', 'Pazar']
day_activity['day_name'] = pd.Categorical(day_activity['day_name'], categories=day_order, ordered=True)
day_activity = day_activity.sort_values('day_name')

plt.figure(figsize=(12, 6))
colors = ['#FF6B6B' if day in ['Saturday', 'Sunday'] else '#4ECDC4' for day in day_activity['day_name']]
plt.bar(day_names_tr, day_activity['post_count'], color=colors, alpha=0.8)
plt.title('Haftanın Günlerine Göre Gönderi Dağılımı', fontsize=16, fontweight='bold')
plt.xlabel('Gün', fontsize=12)
plt.ylabel('Toplam Gönderi Sayısı', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
hourly_activity = df.groupby('hour').size().reset_index(name='post_count')

plt.figure(figsize=(14, 6))
plt.plot(hourly_activity['hour'], hourly_activity['post_count'], 
         marker='o', linewidth=2, markersize=6, color='purple')
plt.fill_between(hourly_activity['hour'], hourly_activity['post_count'], alpha=0.3, color='purple')
plt.title('Saatlere Göre Gönderi Aktivitesi', fontsize=16, fontweight='bold')
plt.xlabel('Saat (UTC)', fontsize=12)
plt.ylabel('Toplam Gönderi Sayısı', fontsize=12)
plt.xticks(range(0, 24))
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Metin Analizi

In [ ]:
if 'title_length' in df.columns:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    axes[0].hist(df['title_length'], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0].set_title('Başlık Uzunluğu Dağılımı', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Karakter Sayısı', fontsize=12)
    axes[0].set_ylabel('Frekans', fontsize=12)
    axes[0].grid(axis='y', alpha=0.3)
    
    axes[1].boxplot(df['title_length'], vert=True)
    axes[1].set_title('Başlık Uzunluğu Box Plot', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Karakter Sayısı', fontsize=12)
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
if 'title' in df.columns:
    all_titles = ' '.join(df['title'].dropna().astype(str))
    
    stopwords = set(['the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 
                     'of', 'with', 'by', 'from', 'as', 'is', 'was', 'are', 'were', 'be',
                     'been', 'being', 'have', 'has', 'had', 'do', 'does', 'did', 'will',
                     'would', 'should', 'could', 'may', 'might', 'must', 'can', 'this',
                     'that', 'these', 'those', 'i', 'you', 'he', 'she', 'it', 'we', 'they'])
    
    wordcloud = WordCloud(width=1600, height=800, 
                         background_color='white',
                         stopwords=stopwords,
                         colormap='viridis',
                         max_words=100,
                         relative_scaling=0.5,
                         min_font_size=10).generate(all_titles)
    
    plt.figure(figsize=(16, 8))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('En Sık Kullanılan Kelimeler (Word Cloud)', fontsize=18, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()

In [ ]:
if 'title' in df.columns:
    all_words = ' '.join(df['title'].dropna().astype(str).str.lower()).split()
    
    filtered_words = [word for word in all_words if word not in stopwords and len(word) > 2]
    
    word_freq = Counter(filtered_words).most_common(20)
    words, counts = zip(*word_freq)
    
    plt.figure(figsize=(12, 8))
    plt.barh(range(len(words)), counts, color='teal', alpha=0.8)
    plt.yticks(range(len(words)), words)
    plt.xlabel('Frekans', fontsize=12)
    plt.title('En Sık Kullanılan 20 Kelime', fontsize=16, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(axis='x', alpha=0.3)
    
    for i, v in enumerate(counts):
        plt.text(v, i, f' {v:,}', va='center', fontsize=10)
    
    plt.tight_layout()
    plt.show()

## 8. Etkileşim Analizi

In [ ]:
if 'score' in df.columns and 'title' in df.columns:
    top_posts = df.nlargest(10, 'score')[['title', 'score', 'num_comments', 'created_utc']]
    print("En Popüler 10 Gönderi (Score'a Göre):")
    print("="*100)
    for idx, row in top_posts.iterrows():
        print(f"\n{row['title'][:80]}...")
        print(f"  Score: {row['score']:,} | Yorumlar: {row['num_comments']:,} | Tarih: {row['created_utc']}")

In [ ]:
if 'score' in df.columns and 'num_comments' in df.columns:
    score_99 = df['score'].quantile(0.99)
    comments_99 = df['num_comments'].quantile(0.99)
    df_filtered = df[(df['score'] <= score_99) & (df['num_comments'] <= comments_99)]
    
    plt.figure(figsize=(12, 6))
    plt.scatter(df_filtered['score'], df_filtered['num_comments'], 
               alpha=0.3, s=10, color='darkblue')
    plt.xlabel('Score', fontsize=12)
    plt.ylabel('Yorum Sayısı', fontsize=12)
    plt.title('Score vs Yorum Sayısı İlişkisi', fontsize=16, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    correlation = df['score'].corr(df['num_comments'])
    plt.text(0.05, 0.95, f'Korelasyon: {correlation:.3f}', 
            transform=plt.gca().transAxes, fontsize=12, 
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
            verticalalignment='top')
    
    plt.tight_layout()
    plt.show()

In [ ]:
if 'score' in df.columns and 'num_comments' in df.columns:
    yearly_engagement = df.groupby('year').agg({
        'score': 'mean',
        'num_comments': 'mean'
    }).reset_index()
    
    fig, ax1 = plt.subplots(figsize=(14, 6))
    
    color1 = 'tab:blue'
    ax1.set_xlabel('Yıl', fontsize=12)
    ax1.set_ylabel('Ortalama Score', color=color1, fontsize=12)
    ax1.plot(yearly_engagement['year'], yearly_engagement['score'], 
            color=color1, marker='o', linewidth=2, markersize=8, label='Score')
    ax1.tick_params(axis='y', labelcolor=color1)
    ax1.grid(True, alpha=0.3)
    
    ax2 = ax1.twinx()
    color2 = 'tab:orange'
    ax2.set_ylabel('Ortalama Yorum Sayısı', color=color2, fontsize=12)
    ax2.plot(yearly_engagement['year'], yearly_engagement['num_comments'], 
            color=color2, marker='s', linewidth=2, markersize=8, label='Yorumlar')
    ax2.tick_params(axis='y', labelcolor=color2)
    
    plt.title('Yıllara Göre Ortalama Etkileşim', fontsize=16, fontweight='bold')
    fig.tight_layout()
    plt.show()

## 9. İleri Düzey Analizler

In [ ]:
df['is_weekend'] = df['day_of_week'].isin([5, 6])  # Cumartesi=5, Pazar=6

weekend_comparison = df.groupby('is_weekend').agg({
    'score': 'mean',
    'num_comments': 'mean',
    'title': 'count'
}).reset_index()
weekend_comparison['is_weekend'] = weekend_comparison['is_weekend'].map({True: 'Hafta Sonu', False: 'Hafta İçi'})
weekend_comparison.columns = ['Dönem', 'Ort. Score', 'Ort. Yorum', 'Gönderi Sayısı']

print("Hafta İçi vs Hafta Sonu Karşılaştırması:")
print("="*60)
print(weekend_comparison.to_string(index=False))

In [ ]:
if 'title_length' in df.columns and 'score' in df.columns:
    df['title_category'] = pd.cut(df['title_length'], 
                                   bins=[0, 50, 100, 150, 200, 1000],
                                   labels=['Çok Kısa (0-50)', 'Kısa (50-100)', 
                                          'Orta (100-150)', 'Uzun (150-200)', 'Çok Uzun (200+)'])
    
    title_length_analysis = df.groupby('title_category').agg({
        'score': 'mean',
        'num_comments': 'mean',
        'title': 'count'
    }).reset_index()
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    axes[0].bar(range(len(title_length_analysis)), title_length_analysis['score'], 
               color='steelblue', alpha=0.8)
    axes[0].set_xticks(range(len(title_length_analysis)))
    axes[0].set_xticklabels(title_length_analysis['title_category'], rotation=45, ha='right')
    axes[0].set_ylabel('Ortalama Score', fontsize=12)
    axes[0].set_title('Başlık Uzunluğu vs Ortalama Score', fontsize=14, fontweight='bold')
    axes[0].grid(axis='y', alpha=0.3)
    
    axes[1].bar(range(len(title_length_analysis)), title_length_analysis['title'], 
               color='coral', alpha=0.8)
    axes[1].set_xticks(range(len(title_length_analysis)))
    axes[1].set_xticklabels(title_length_analysis['title_category'], rotation=45, ha='right')
    axes[1].set_ylabel('Gönderi Sayısı', fontsize=12)
    axes[1].set_title('Başlık Uzunluğu Dağılımı', fontsize=14, fontweight='bold')
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 10. Özet ve Bulgular

In [ ]:
print("="*80)
print("PROJE ÖZETİ VE ANA BULGULAR")
print("="*80)

print("\n📊 VERİ SETİ ÖZETİ:")
print(f"  • Toplam gönderi: {len(df):,}")
print(f"  • Tarih aralığı: {df['created_utc'].min().year} - {df['created_utc'].max().year}")
print(f"  • Analiz edilen yıl sayısı: {df['year'].nunique()}")

print("\n📈 ZAMAN SERİSİ BULGULARI:")
most_active_year = yearly_posts.loc[yearly_posts['post_count'].idxmax()]
print(f"  • En aktif yıl: {int(most_active_year['year'])} ({most_active_year['post_count']:,} gönderi)")
most_active_month = monthly_activity.loc[monthly_activity['post_count'].idxmax()]
print(f"  • En aktif ay: {most_active_month['month_name']} ({most_active_month['post_count']:,} gönderi)")
most_active_day = day_activity.loc[day_activity['post_count'].idxmax()]
print(f"  • En aktif gün: {day_names_tr[day_order.index(most_active_day['day_name'])]}")

if 'score' in df.columns:
    print("\n💬 ETKİLEŞİM BULGULARI:")
    print(f"  • Ortalama score: {df['score'].mean():.2f}")
    print(f"  • Medyan score: {df['score'].median():.2f}")
    print(f"  • En yüksek score: {df['score'].max():,}")
    if 'num_comments' in df.columns:
        print(f"  • Ortalama yorum: {df['num_comments'].mean():.2f}")
        print(f"  • Score-Yorum korelasyonu: {df['score'].corr(df['num_comments']):.3f}")

if 'title_length' in df.columns:
    print("\n📝 METİN ANALİZİ BULGULARI:")
    print(f"  • Ortalama başlık uzunluğu: {df['title_length'].mean():.1f} karakter")
    print(f"  • Ortalama kelime sayısı: {df['title_word_count'].mean():.1f} kelime")
    print(f"  • En uzun başlık: {df['title_length'].max()} karakter")
    print(f"  • En kısa başlık: {df['title_length'].min()} karakter")

print("\n" + "="*80)
print("✓ Analiz tamamlandı!")
print("="*80)

## 11. Sonuç ve Öneriler

### Proje Sonuçları:

Bu analizde Reddit r/news subreddit'inin 2008-2024 yılları arasındaki verileri incelendi. Aşağıdaki ana bulgulara ulaşıldı:

1. **Zaman Trendleri**: Yıllar içinde gönderi sayısında belirgin değişimler gözlemlendi
2. **Aktivite Paternleri**: Belirli günler ve saatlerde daha yüksek aktivite görüldü
3. **Etkileşim Analizi**: Score ve yorum sayısı arasında pozitif korelasyon bulundu
4. **Metin Özellikleri**: Başlık uzunluğu ve kelime kullanımı paternleri belirlendi

### Gelecek Çalışmalar İçin Öneriler:

- Sentiment analizi ile pozitif/negatif haber trendlerini inceleme
- Topic modeling ile ana haber kategorilerini belirleme
- Makine öğrenmesi ile viral içerik tahmini
- Önemli olayların (seçimler, pandemi vb.) etkisini detaylı analiz etme

---

**Proje Sahibi**: Veri Bilimine Giriş Dersi "J" Grubu 
**Tarih**: 2025  
**Veri Kaynağı**: Kaggle - Reddit r/news Dataset (2008-2024)